# Preprocesado de datos

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

In [2]:
__RAW_DATA_PATH__ = "../data/raw"
__INTERIM_DATA_PATH__ = "../data/interim"

__OUTPUT_DF_NAME__ = "easy_money_prep_1"

__PREDICT_SALARY__ = False
__DO_ONE_HOT_ENCODE__ = False

# CONFIG
# pd.options.display.float_format = '${:,.2f}'.format

### FUNCTIONS

In [3]:
def one_hot_encode(dataframe: pd.DataFrame, column_name: str) -> pd.DataFrame:
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    return pd.concat([dataframe, _dummy_dataset], axis=1).drop([column_name], axis=1)


def encode_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:
    cat_columns = [col for col in dataframe.select_dtypes(exclude=[np.number, "datetime"]).columns]
    print(f"columns to encode: {cat_columns}")

    print(f"dataframe before one-hot encoding: {dataframe.shape}")
    for cat_col in cat_columns:
        dataframe = one_hot_encode(dataframe, cat_col)

    print(f"dataframe after one-hot encoding:  {dataframe.shape}")
    return dataframe


def export_to_csv(dataframe: pd.DataFrame, filename: str, path: str = __INTERIM_DATA_PATH__):
    exported_path = f"{path}/{filename}.csv"
    dataframe.to_csv(exported_path)
    print(f"Dataframe exported to {exported_path}")


def predict_salaries(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    predict the salary of clients who have NAN salaries using KNNImputer,
    based on the closest values as a function of age
    :param dataframe: dataframe to used
    :return: salary_df
    """
    _salary = dataframe.copy(deep=True).groupby("pk_cid").agg(
        age=("age", np.max),
        salary=("salary", np.mean),
    ).sort_values("age")
    _salary.reset_index(inplace=True)

    columns_to_evaluate = ["age", "salary"]

    is_null_condition = _salary["salary"].isnull()
    is_outlier_condition = _salary["salary"] <= np.quantile(_salary[_salary["salary"].notnull()]["salary"], q=0.99)
    salary_train_df = _salary[is_null_condition | is_outlier_condition]

    print("Predicting salary...")
    imputer = KNNImputer(n_neighbors=2)
    imputer.fit(salary_train_df[columns_to_evaluate])
    prediction = imputer.transform(_salary[columns_to_evaluate])
    salary_predict = pd.DataFrame(prediction, columns=["age", "salary_prediction"])

    _salary["knn_salary"] = salary_predict["salary_prediction"]
    return _salary

## Import data

In [4]:
# Load products_df
products_df = pd.read_csv(f"{__RAW_DATA_PATH__}/products_df.csv")
del (products_df["Unnamed: 0"])
print(f"* products_df loaded           *  shape: {products_df.shape}")

# Load commercial_activity_df
commercial_activity_df = pd.read_csv(f"{__RAW_DATA_PATH__}/commercial_activity_df.csv")
del (commercial_activity_df["Unnamed: 0"])
print(f"* commercial_activity_dfloaded *  shape: {commercial_activity_df.shape}")

# Load sociodemographic_df
sociodemographic_df = pd.read_csv(f"{__RAW_DATA_PATH__}/sociodemographic_df.csv")
del (sociodemographic_df["Unnamed: 0"])
print(f"* sociodemographic_df loaded   *  shape: {sociodemographic_df.shape}")

* products_df loaded           *  shape: (5962924, 17)
* commercial_activity_dfloaded *  shape: (5962924, 6)
* sociodemographic_df loaded   *  shape: (5962924, 8)


## Merge Dataframes

In [5]:
_df = pd.merge(left=commercial_activity_df, right=sociodemographic_df, how="inner", on=["pk_cid", "pk_partition"])
df = pd.merge(left=_df, right=products_df, how="inner", on=["pk_cid", "pk_partition"])
print(f"* Dataframes merged *  shape: {df.shape}")

* Dataframes merged *  shape: (5962924, 27)


## Data Preprocessing

In [6]:
df_prep = df.copy(deep=True)

##### pk_partition

In [7]:
df_prep["pk_partition"] = pd.to_datetime(df_prep["pk_partition"])

##### entry_date

In [8]:
# Obteníamos un error al convertir nuestra variable a fecha debido a las fechas correspondientes
# al 29 de Febrero de los años 2015 y 2019 puesto que no existen.
# Por este motivo, decidimos reemplazar estos valores por la fecha más cercana (día anterior)
df_prep["entry_date"] = df_prep["entry_date"].replace("2015-02-29", "2015-02-28")
df_prep["entry_date"] = df_prep["entry_date"].replace("2019-02-29", "2019-02-28")

df_prep["entry_date"] = pd.to_datetime(df_prep["entry_date"])

#Generamos atributos extras a partir de entry_date
df_prep["entry_date_month"] = df_prep["entry_date"].dt.month.astype(int)
df_prep["entry_date_year"] = df_prep["entry_date"].dt.year.astype(int)
df_prep["entry_date_day"] = df_prep["entry_date"].dt.day.astype(int)
df_prep["entry_date_week"] = df_prep["entry_date"].dt.isocalendar().week.astype(int)
df_prep["entry_date_weekday"] = df_prep["entry_date"].dt.weekday.astype(int)
df_prep["entry_date_trim"] = df_prep["entry_date"].dt.quarter.astype(int)

##### entry_channel

In [9]:
df_prep["entry_channel"].fillna("OTROS", inplace=True)
df_prep["entry_channel"].value_counts(dropna=False)

KHE    3113947
KFC     890620
KHQ     590280
KAT     416084
KHK     230197
        ...   
KEJ          8
KHS          5
KDA          2
KFP          2
KDS          1
Name: entry_channel, Length: 69, dtype: int64

##### active_customer

In [10]:
df_prep["active_customer"] = df_prep["active_customer"].astype(int)

##### segment

In [11]:
df_prep["segment"].fillna("00 - OTROS", inplace=True)
df_prep["segment"].value_counts(dropna=False)

03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
00 - OTROS             133944
01 - TOP                97939
Name: segment, dtype: int64

##### region_code

In [12]:
df_prep["region_code"] = df_prep["region_code"].fillna(-1).astype(int)
df_prep["region_code"].value_counts(dropna=False)

 28    1185757
 8      600362
 46     358142
 30     296856
 41     290447
 15     254891
 29     187911
 36     175850
 3      171358
 11     160894
 33     134562
 50     132983
 6      122587
 35     121982
 47     116948
 18     102789
 45     102328
 37      92538
 10      83283
 14      75405
 2       72071
 21      72037
 13      70092
 39      69589
 12      68709
 7       62436
 32      54639
 27      52783
 43      51782
 17      50749
 9       47746
 25      46887
 16      36094
 24      35426
 48      35256
 26      34790
 4       32680
 31      31241
 49      30965
 23      30955
 38      29001
 34      27545
 19      26762
 40      21658
 5       21300
 22      21112
 20      18833
 1       12999
 44      12616
 42       8500
 51       2896
 52       2638
-1        2264
Name: region_code, dtype: int64

##### gender

In [13]:
df_prep["gender"].fillna("non-binary", inplace=True)
df_prep["gender"].replace("H", "female", inplace=True)
df_prep["gender"].replace("V", "male", inplace=True)
df_prep["gender"].value_counts(dropna=False)

female        3087502
male          2875397
non-binary         25
Name: gender, dtype: int64

##### deceased

In [14]:
df_prep["deceased"].replace("N", 0, inplace=True)
df_prep["deceased"].replace("S", 1, inplace=True)
df_prep["deceased"].value_counts()

0    5961849
1       1075
Name: deceased, dtype: int64

##### salary

In [15]:
%%time
if __PREDICT_SALARY__:
    salary_df = predict_salaries(df_prep)
    salary_df.to_csv(f"{__INTERIM_DATA_PATH__}/salary_df.csv")
else:
    salary_df = pd.read_csv(f"{__INTERIM_DATA_PATH__}/salary_df.csv", index_col=0)

print(f"salary_df.shape: {salary_df.shape}")
salary_df.head(500)

Predicting salary...
salary_df.shape: (456373, 4)
Wall time: 59min 37s


,pk_cid,age,salary,knn_salary
0,1550397,2,NaN,177539.28
1,1526522,2,NaN,177539.28
2,1419718,2,178055.85,178055.85
3,1513456,2,NaN,177539.28
4,1539094,2,NaN,177539.28
...,...,...,...,...
495,1114054,5,555357.33,555357.33
496,1519470,5,NaN,93842.22
497,1548937,5,NaN,93842.22
498,1298232,5,31383.93,31383.93


In [16]:
salaries_reshaped = pd.merge(
    left=df_prep[["pk_cid", "salary"]],
    right=salary_df[["pk_cid", "knn_salary"]],
    on="pk_cid",
    how="left"
)
df_prep["salary"] = df_prep["salary"].fillna(salaries_reshaped["knn_salary"])

##### payroll and payroll_account

In [17]:
df_prep.dropna(subset=["payroll", "payroll_account"], inplace=True)
df_prep["payroll"] = df_prep["payroll"].astype(int)
df_prep["payroll"].value_counts(dropna=False)

0    5757076
1     205787
Name: payroll, dtype: int64

##### pension_plan

In [18]:
df_prep["pension_plan"] = df_prep["pension_plan"].astype(int)

## Export Dataframe

In [19]:
df_prep_not_null = df_prep.copy(deep=True).sort_values(["pk_cid", "pk_partition"]).reset_index(drop=True)

In [20]:
%%time
if __DO_ONE_HOT_ENCODE__:
    df_prep_one_hot = encode_dataframe(df_prep_not_null)
    export_to_csv(df_prep_one_hot, __OUTPUT_DF_NAME__)
else:
    export_to_csv(df_prep_not_null, __OUTPUT_DF_NAME__)

columns to encode: ['entry_channel', 'segment', 'country_id', 'gender']
dataframe before one-hot encoding: (5962863, 33)
dataframe after one-hot encoding:  (5962863, 146)
Dataframe exported to ../data/interim/easy_money_prep_1.csv
Wall time: 3min 37s
